In [10]:
import csv

In [8]:
PASSAGE_PATH = "/home/jianx/data/passage_indices.csv"
QUERY_TRAIN_PATH = "/home/jianx/data/queries_train_indices.csv"
QUERY_EVAL_PATH = "/home/jianx/data/queries_eval_indices.csv"
QUERY_DEV_PATH = "/home/jianx/data/queries_dev_indices.csv"
TRAIN_TRIPLE_PATH = "/home/jianx/data/qidpidtriples.train.full.2.tsv"

In [6]:
def id_content_reader(path):
    with open(path) as f:
        reader = csv.reader(f)
        result = {}
        for row in reader:
            result[int(row[0])] = list(map(int, row[1].split()))
        return result

In [6]:
def pos_neg_dict_reader(path):
    with open(path) as file:
        line = file.readline()
        pos_neg_dict = {}
        while line:
            ids = list(map(int, line.split()))
            pos_neg_dict[ids[0]] = [[ids[1], ids[2]]]
            current_id = ids[0]
            line = file.readline()
            ids = list(map(int, line.split()))
            while line and current_id == ids[0]:
                pos_neg_dict[current_id].append([int(ids[1]), int(ids[2])])
                line = file.readline()
                ids = list(map(int, line.split()))
    return pos_neg_dict

In [7]:
positive_negative = pos_neg_dict_reader(TRAIN_TRIPLE_PATH)

In [11]:
passages = id_content_reader(PASSAGE_PATH)
queries_train = id_content_reader(QUERY_TRAIN_PATH)
queries_eval = id_content_reader(QUERY_EVAL_PATH)
queries_dev = id_content_reader(QUERY_DEV_PATH)

In [20]:
import pickle

def obj_reader(path):
    with open(path, 'rb') as handle:
        return pickle.load(handle, encoding="bytes")


def obj_writer(obj, path):
    with open(path, 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
obj_writer(passages, "/home/jianx/data/passages.dict")
obj_writer(queries_train, "/home/jianx/data/queries_train.dict")
obj_writer(queries_eval, "/home/jianx/data/queries_eval.dict")
obj_writer(queries_dev, "/home/jianx/data/queries_dev.dict")

In [11]:
queries_test = id_content_reader("/home/jianx/data/queries_test_indices.csv")
obj_writer(queries_test, "/home/jianx/data/queries_test.dict")

In [12]:
obj_writer(positive_negative, "/home/jianx/data/qid_pos_neg.dict")

In [15]:
def initial_ranking_reader(path):
    initial_ranking_dict = {}
    with open(path) as file:
        line = file.readline()
        while line:
            tokens = line.split()
            qid = int(tokens[0])
            pid = int(tokens[1])
            if qid not in initial_ranking_dict:
                initial_ranking_dict[qid] = []
            initial_ranking_dict[qid].append(pid)
            line = file.readline()
    return initial_ranking_dict

In [16]:
initial_ranking = initial_ranking_reader("/home/jianx/data/msmarco-passagetest2019-top1000.tsv")
initial_ranking[855410]

[8651770, 8651772, 8651775, 8651776, 8651771]

In [17]:
obj_writer(initial_ranking, "/home/jianx/data/initial_ranking.dict")
print(len(initial_ranking))

200


In [18]:
def rel_reader(path):
    with open(path) as file:
        line = file.readline()
        rel_dict = {}
        while line:
            tokens = line.split()
            qid = int(tokens[0])
            pid = int(tokens[2])
            score = int(tokens[3])
            if qid not in rel_dict:
                rel_dict[qid] = {}
            if score != 0:
                rel_dict[qid][pid] = score
            line = file.readline()

    return rel_dict

In [19]:
rel_scores = rel_reader("/home/jianx/data/2019qrels-pass.txt")
obj_writer(rel_scores, "/home/jianx/data/rel_scores.dict")
print(rel_scores)

{19335: {1720389: 1, 1720395: 1, 1729: 2, 2046505: 1, 3045565: 1, 3045567: 1, 3175481: 3, 3175484: 3, 527690: 1, 527692: 1, 527697: 1, 6452949: 1, 7122355: 1, 7320614: 1, 819168: 1, 8412681: 2, 8412682: 3, 8412683: 2, 8412684: 3, 8412685: 1}, 47923: {1200258: 2, 1296110: 1, 1300989: 1, 1354790: 1, 1507610: 1, 1508658: 1, 1669816: 1, 1681331: 2, 1681332: 1, 1681334: 2, 1681335: 1, 1681336: 1, 1681337: 1, 1681339: 1, 1767240: 1, 1868437: 2, 1868438: 1, 1923153: 1, 1960681: 1, 2044900: 1, 2148997: 1, 2157582: 1, 2477210: 2, 2477211: 1, 2655670: 2, 2719555: 1, 2749477: 1, 2772864: 1, 284591: 2, 2917510: 1, 320920: 1, 3246976: 1, 3348837: 1, 3348838: 1, 3630933: 1, 3630935: 2, 3694265: 1, 3748434: 2, 3986046: 1, 4081982: 1, 4143232: 2, 4257346: 1, 4297620: 1, 4321228: 1, 4321230: 2, 4416477: 1, 4652795: 2, 4684891: 1, 4684893: 2, 4684895: 1, 473807: 3, 473809: 1, 4778734: 1, 4793758: 1, 4997370: 2, 5032362: 1, 5051722: 1, 5053382: 1, 5055000: 2, 5137402: 1, 5147527: 1, 5147529: 1, 5147530: 

In [20]:
avg_len = 0
for qid in rel_scores:
    print(len(rel_scores[qid]))
    avg_len += len(rel_scores[qid])
print(avg_len/len(rel_scores))

20
112
83
81
141
28
64
23
101
133
289
53
229
24
211
56
35
94
154
96
55
75
69
75
4
92
25
13
276
31
60
35
142
77
52
341
24
119
46
12
139
28
285
95.3953488372093


In [21]:
POS_NEG_DICT_PATH = "/home/jianx/data/qid_pos_neg.dict"
PASSAGE_DICT_PATH = "/home/jianx/data/passages.dict"
QUERY_TRAIN_DICT_PATH = "/home/jianx/data/queries_train.dict"
QUERY_EVAL_DICT_PATH = "/home/jianx/data/queries_eval.dict"
QUERY_DEV_DICT_PATH = "/home/jianx/data/queries_dev.dict"
TOP_DICT_PATH = "/home/jianx/data/initial_ranking.dict"
RATING_DICT_PATH = "/home/jianx/data/rel_scores.dict"
QUERY_TEST_DICT_PATH = "/home/jianx/data/queries_test.dict"
import time

start = time.time()
print(start)
def load():
    pos_neg_dict = obj_reader(POS_NEG_DICT_PATH)
    query_dict = obj_reader(QUERY_TRAIN_DICT_PATH)
    passage_dict = obj_reader(PASSAGE_DICT_PATH)
    top_dict = obj_reader(TOP_DICT_PATH)
    rating_dict = obj_reader(RATING_DICT_PATH)
    query_test_dict = obj_reader(QUERY_TEST_DICT_PATH)
    return pos_neg_dict, query_dict, passage_dict, top_dict, rating_dict, query_test_dict
load()
print(time.time() - start)

1594004119.019143
479.03558897972107


1593675778.5683913